In [1]:
import sys
sys.path.insert(0, "../")

from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from grad_em import *

N = 10
K = 2
m = 500
device = "cpu"
true_r = (6 * torch.rand(N, device=device) - 3).clone()
true_r = true_r - true_r.mean()
true_beta = torch.rand(K, device=device)
print(true_r)
print(true_beta)

sample_fn, flat_probs = logistic_preference_dist(true_r, true_beta)
samples = sample_fn(m)


# --- Example Usage ---
# Data format: [winner_id, loser_id, annotator_id]
raw_data = samples

# Convert to tensors
winners = torch.tensor([d[0] for d in raw_data])
losers = torch.tensor([d[1] for d in raw_data])
annotators = torch.tensor([d[2] for d in raw_data])
data_tensors = (winners, losers, annotators)

model = GradientEM(N, K)

# Separate optimizers for alternating updates
opt_r = torch.optim.Adam(model.item_rewards.parameters(), lr=0.01)
opt_beta = torch.optim.Adam(model.worker_betas.parameters(), lr=0.01)

r, beta = train_with_convergence(model, data_tensors, opt_r, opt_beta)

print("\nEstimated Rewards:", model.item_rewards.weight.data.flatten())
print("Estimated Competencies:", model.worker_betas.weight.data.flatten())

tensor([-2.5595, -0.4848,  2.2301, -1.0132, -1.0835,  2.8000, -2.1424,  2.0619,
         0.7193, -0.5279])
tensor([0.2697, 0.0983])


 42%|██████████████▌                    | 417/1000 [00:00<00:01, 519.91it/s]


Estimated Rewards: tensor([-1.8672,  0.3399,  2.0280,  0.0092, -1.3391,  1.4722, -0.4436,  0.5531,
         0.2115, -0.9641])
Estimated Competencies: tensor([0.3617, 0.1464])


In [2]:
import os
import csv
import gc
import random
from collections import defaultdict
import sys
import numpy as np
import pandas as pd
import torch
from scipy.stats import kendalltau


from pgem import EMWrapper
from metrics import compute_acc, compute_weighted_acc
import opt_fair
from crowdkit.aggregation import NoisyBradleyTerry
import choix
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 53%|██████████████████▋                | 533/1000 [00:03<00:02, 158.95it/s]


N=10, K=2, GradEM_acc=0.9667±0.0509


100%|██████████████████████████████████| 1000/1000 [00:04<00:00, 242.78it/s]



Reached max_epochs without full convergence.


 32%|███████████▎                       | 323/1000 [00:01<00:03, 180.95it/s]


KeyboardInterrupt: 